In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
device = torch.device("cpu")

In [ ]:
N=1000
s=np.linspace(0,10,N)
y1=np.sin(s)
y2=np.cos(s)
plt.plot(y1)
plt.plot(y2)

In [ ]:
def update_identity_rho(I, mu):
    zero = torch.tensor(0.0, dtype=torch.float64)
    for i in range(n_weights):
        if (-x[i]) < zero and torch.isclose(mu[i], zero):
            I[i,i] = 0
        else:
            I[i,i] = rho

In [ ]:
def update_dual_variables(lmbda, mu):
    with torch.no_grad():
        lmbda = lmbda + rho*(x.sum()-1)
        mu = torch.maximum(torch.zeros_like(x), mu+rho*(-x)) 

In [ ]:
def are_kkt_conditions_verified(atol=1e-4):
    # dx L = 0
    dx = torch.autograd.grad(lf(x, lmbda, mu), x)[0]
    if torch.isclose(dx, torch.zeros_like(dx), atol=atol).all():
        # c(x) = 0 | x.sum()-1 = 0
        if torch.isclose((x.sum()-1), torch.tensor(0.0, dtype=torch.float64), atol=atol):
            # h(x) <= 0 | (-x) <= 0
            if ((-x) <= 0.0).all():
                # mu >= 0
                if (mu >= 0.0).all():
                    # mu*.h(x) = 0 
                    if torch.isclose((-x)*mu, torch.zeros_like(mu), atol=atol).all():
                        return True
                    
    return False

In [ ]:
def f(x, loss_fn="Var", alp=None):
    y = torch.matmul(x,Y)
    if loss_fn == "Var":        
        return y.var()
    elif loss_fn == "Cov":
        cov_matrix = y.cov()
        return torch.triu(cov_matrix, diagonal=1).sum()
    elif loss_fn == "Mean+Var":
        return alp*y.mean() + (1-alp)*y.var()
    elif loss_fn == "Mean+Cov":
        cov_matrix = y.cov()
        return alp*y.mean() + (1-alp)*torch.triu(cov_matrix, diagonal=1).sum()

def lf(x, lmbda, mu, loss_fn="Var"):
    return f(x) + lmbda * (x.sum() - 1) + torch.matmul(-x,mu)

def lf_rho(x, lmbda, mu, rho, loss_fn="Var"):
    return lf(x,lmbda,mu) + rho/2*(x.sum()-1)**2 + 1/2*torch.matmul(torch.matmul(-x,I_rho),(-x))

In [ ]:
Y = np.stack((y1,y2),axis=-1).T

n_weights = Y.shape[0]
rho = torch.tensor(3.0, dtype=torch.float64)

Y=torch.from_numpy(Y)
Y=Y.to(device)

x = np.random.rand(n_weights)
x=x/x.sum(axis=-1)
x = torch.from_numpy(x)
x.requires_grad = True
x = x.to(device)

lmbda = torch.tensor(0.5, requires_grad=True, device=device)
mu = torch.tensor([0.5 for i in range(n_weights)], requires_grad=True, dtype=torch.float64, device=device)

I_rho = np.eye(Y.shape[0])
I_rho = torch.from_numpy(I_rho)*rho
I_rho = I_rho.to(device)

objs = []
xs = [x.cpu().detach().numpy().copy()]
lmbdas = [lmbda.item()]
mus = [mu.cpu().detach().numpy().copy()]

rho_scaling = torch.tensor(1.1,dtype=torch.float64)
step_size = torch.tensor(1e-3, dtype=torch.float64)
rhos = [rho.item()]

In [ ]:
%%time
n_steps = 250
n_iterations = 40
for it in range(n_iterations):
    # solve for current lagrangian multipliers
    for i in range(n_steps):
        obj = lf_rho(x, lmbda, mu, rho, loss_fn="Cov")
        dx = torch.autograd.grad(obj, x)
        with torch.no_grad():
            x -= step_size * dx[0]
        xs.append(x.cpu().detach().numpy().copy())
        objs.append([obj.item(), lf(x, lmbda, mu).item(), f(x).item()])
    objs.append([lf_rho(x,lmbda, mu, rho).item(), lf(x, lmbda, mu).item(), f(x).item()])    

    mus.append(mu.cpu().detach().numpy().copy())
    lmbdas.append(lmbda.item())
    # Update lagrangian multipliers and rho
    with torch.no_grad():
        lmbda = lmbda + rho*(x.sum()-1)
        mu = torch.maximum(torch.zeros_like(x), mu+rho*(-x)) 
    update_identity_rho(I_rho, mu)
        
    rho = rho*rho_scaling
    rhos.append(rho.item())
    
    # Assert KKT Conditions
    converged = are_kkt_conditions_verified()
    
    if converged:
        break

In [ ]:
plt.plot([i[0] for i in objs], label="lf_rho")
plt.plot([i[1] for i in objs], label="lf")
plt.plot([i[2] for i in objs], label="f")
plt.legend()

In [ ]:
plt.plot(xs);
print(xs[-1])

In [ ]:
plt.plot(rhos)

In [ ]:
are_kkt_conditions_verified()

# Get wind data

In [ ]:
from getpass import getpass

In [ ]:
from io import StringIO

In [ ]:
from datetime import datetime, timedelta
from azure.storage.fileshare import ShareServiceClient, ShareDirectoryClient

In [ ]:
from configparser import RawConfigParser

In [ ]:
try:
    config = RawConfigParser()
    config.read("../config.ini")
    sas_token_url = config["File Storage"]["sas_token"]
except:
    #url to the root file share folder ("data")
    sas_token_url = getpass("sas taken and url: ")

In [ ]:
share_service_client = ShareServiceClient(account_url=sas_token_url)

In [ ]:
dir_client = ShareDirectoryClient(account_url=sas_token_url, directory_path="data", share_name="wind-covariation")

In [ ]:
[i["name"] for i in dir_client.list_directories_and_files()]

In [ ]:
file_client = dir_client.get_file_client("offshore_wind_locations.csv")
df = pd.read_csv(StringIO(file_client.download_file().content_as_text()))
df

In [ ]:
dir_client = ShareDirectoryClient(account_url=sas_token_url, directory_path="data/nve/profiler/Wind and solar",
                                  share_name="wind-covariation")

In [ ]:
[i["name"] for i in dir_client.list_directories_and_files()]

# Load modules

In [ ]:
from geopy.distance import geodesic
from scipy.optimize import curve_fit

In [ ]:
import seaborn as sns

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

import sys 
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from Wind.analyze import (
    get_corr_figure,
    get_hours_shift_figure,
    get_mean_std_wind_figure,
    get_corr_distance_figure,
    get_line_plot_with_mean,
    get_histogram_2d_figure,
    get_scatter_2d_figure,
    get_scatter_with_kernel_density_2d_figure,
    get_scatter_density_2d_figure,
)

In [ ]:
dir_client = ShareDirectoryClient(account_url=sas_token_url, directory_path="data", share_name="wind-covariation")

file_client = dir_client.get_file_client("offshore_wind_locations.csv")
df_wind_locations = pd.read_csv(StringIO(file_client.download_file().content_as_text()))

file_client = dir_client.get_file_client("nve_offshore_wind_areas.csv")
df_nve_wind_locations = pd.read_csv(StringIO(file_client.download_file().content_as_text()))

df_nve_wind_locations = df_nve_wind_locations.sort_values(by="lat")  # Sort by south to north

df_locations = pd.concat([df_wind_locations, df_nve_wind_locations], axis=0)
df_locations = df_locations.reset_index(drop=True)
df_locations = df_locations.sort_values(by="lat")  # Sort by south to north

In [ ]:
# Plot locations on map
fig = px.scatter_mapbox(
    df_locations,
    lat="lat",
    lon="lon",
    color="location",
    zoom=3,
    size_max=10,
    height=600,
    size=[3 for _ in df_locations.iterrows()],
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show(config=dict(editable=True))

In [ ]:
df_locations

In [ ]:
dir_client = ShareDirectoryClient(
    account_url=sas_token_url,
    directory_path="data", share_name="wind-covariation"
)

# Load data
data = []
for l in df_locations["location"].values:
    file_client = dir_client.get_file_client(f"{l}.csv")
    df_temp = pd.read_csv(StringIO(file_client.download_file().content_as_text()), index_col=0, parse_dates=True)
#     pd.read_csv(f"data/{l}.csv", index_col=0, parse_dates=True)
    data.append(df_temp)

df = pd.concat(data, axis=1)
df = df[df_locations["location"]]  # Sort by south to north

df.info()
df.describe()

In [ ]:
fig = get_corr_figure(df)
fig.show()

fig = get_corr_distance_figure(df, df_locations)
fig.show()

In [ ]:
locs = []
for _, row in df_locations.iterrows():
    locs.append((row["lat"], row["lon"]))

distances = []
for i in locs:
    distances.append([])
    for j in locs:
        distances[-1].append(geodesic(i, j).km)

df_corr = df.corr()
mask = np.triu(np.ones_like(df_corr, dtype=bool))
df_corr = df_corr.mask(mask).round(2)

df_distance = pd.DataFrame(data=distances, index=df_corr.index, columns=df_corr.columns)
mask = np.triu(np.ones_like(df_distance, dtype=bool))
df_distance = df_distance.mask(mask).round(2)

corr_list = []
for i, row in df_corr.iterrows():
    for j, v in row.items():
        if not np.isnan(v):
            corr_list.append((f"{i} <-> {j}", v))
dist_list = []
for i, row in df_distance.iterrows():
    for j, v in row.items():
        if not np.isnan(v):
            dist_list.append((f"{i} <-> {j}", v))

df_temp = pd.DataFrame(corr_list)
df_temp = df_temp.set_index(0)
df_temp = df_temp.rename(columns={1: "Correlation"})

df_temp1 = pd.DataFrame(dist_list)
df_temp1 = df_temp1.set_index(0)
df_temp1 = df_temp1.rename(columns={1: "Distance [km]"})

df_corr_dist = pd.concat([df_temp, df_temp1], axis=1)
df_corr_dist = df_corr_dist.reset_index()
df_corr_dist = df_corr_dist.rename(columns={0: "Span"})

# Fit an exponential function
def func(x, a, b, c):
    return a * np.exp(-b * x) + c

popt, pcov = curve_fit(
    func, df_corr_dist["Distance [km]"].values, df_corr_dist["Correlation"].values, p0=[1, 0.005, 0]
)

xn = np.linspace(df_corr_dist["Distance [km]"].min(), df_corr_dist["Distance [km]"].max(), 2500)

In [ ]:
import plotly.graph_objects as go

In [ ]:
my_template = go.layout.Template()

my_template.layout.legend = dict(yanchor="top", xanchor="right", y = 0.95, x=0.95,
                              bgcolor='rgba(255, 255, 255, 0.8)', bordercolor='black', borderwidth=1,
                                font=dict(size=12, family='Times New Roman'))

my_template.layout.xaxis = dict(
    zeroline=False, zerolinewidth=1, zerolinecolor='Black', mirror='all',
    showgrid=False, ticks='inside', showline=True, tickfont=dict(size=14, family='Times New Roman'),
    title_font=dict(size=14, family='Times New Roman'))
my_template.layout.yaxis = dict(
    zeroline=False, zerolinewidth=1, zerolinecolor='Black', mirror='all',
    showgrid=False, ticks='inside', showline=True, tickfont=dict(size=14, family='Times New Roman'),
    title_font=dict(size=14, family='Times New Roman'))
my_template.layout.margin = dict(l=70, r=10, b=50, t=10)
my_template.layout.width = 700
my_template.layout.height = 400

data = [
    go.Scatter(
        x=df_corr_dist["Distance [km]"],
        y=df_corr_dist["Correlation"],
        text=df_corr_dist["Span"],
#         marker_color="Black",
        marker=dict(
            color="Black",
            size=5,
            line=dict(width=0)
        ),
        mode="markers",
        name="Between two wind farms",
    ),
    go.Scatter(
        x=xn,
        y=func(xn, *popt),
#         marker_color=,
#         marker_linewidth=2,
        line=dict(
            color="#5D8CC0",
            width=3
        ),
        name=r"$1.05 \exp(\frac{-1}{490.4}x) + 0.02$",
    ),
]

fig = go.Figure(data=data)

fig.update_layout(
    template=my_template,
    title=f"",
    xaxis_title='Distance [km]',
    yaxis_title='Correlation [-]'
)
fig.show()
fig.write_image("../images/corr-distance.pdf")

In [ ]:
f"{popt[0]:.2f} - {1/popt[1]:.1f} - {popt[2]:.2f}"

In [ ]:
df.columns

In [ ]:
df_nve_wind_locations["location"]

In [ ]:
froya_lat = df_nve_wind_locations[df_nve_wind_locations["location"]=="Frøyabanken"]["lat"].values[0]

cols_south = df_nve_wind_locations[df_nve_wind_locations["lat"]<froya_lat]["location"].to_list()
cols_north = df_nve_wind_locations[df_nve_wind_locations["lat"]>=froya_lat]["location"].to_list()

In [ ]:
df[cols_north]

In [ ]:
n_shifts = 24
quantile = 0.99
df["All 15 wind farms"] = df.mean(axis=1)
df["Farms north of Stadt"] = df[cols_north].mean(axis=1)
df["Farms south of Stadt"] = df[cols_south].mean(axis=1)

df_t = df[["Utsira nord", "Sørlige Nordsjø I", "Nordmela", "Farms south of Stadt","Farms north of Stadt","All 15 wind farms"]]
df_shift = pd.concat([df_t.diff(i).quantile(q=quantile) for i in range(n_shifts)], axis=1).T

df_shift.index = [i for i in range(n_shifts)]

df_shift.index.name = "hour shift"
n_cols = len(df_shift.columns)
colormap = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c']
# colormap = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462']
# colormap = sns.color_palette("Set3", n_cols).as_hex()
colors = {col: color for col, color in zip(df_shift.columns, colormap)}

fig = df_shift.plot(
    title=f"", template=my_template, labels=dict(value="Absolute change in power output"), color_discrete_map=colors
)

df = df.drop(columns=["All 15 wind farms", "Farms north of Stadt", "Farms south of Stadt"])

fig.update_layout(
    legend_title="",
    legend=dict(y = 0.95, x=0.30),
    yaxis_range = [0,1]                
)

fig.write_image(f"../images/shift-quantile{quantile}.pdf")
fig.show()

In [ ]:
df.columns

In [ ]:
colors=sns.color_palette("mako",5)

In [ ]:
colors

# Line plot

In [ ]:
area = "Utsira nord"
resample_period='7D'

ind = df.index.map(lambda x: x - pd.Timestamp(year=x.year, month=1, day=1, hour=0, minute=0))

dff = pd.DataFrame(df[area].values, index=ind, columns=[area])
dff["Year"] = df.index.year
dff = dff.reset_index().pivot(index="index", columns="Year").dropna()
dff.columns = dff.columns.droplevel()
dff.index = dff.index + pd.Timestamp("1970-1-1")  # Hack to show dates in figure
years = dff.columns

dff = dff.resample(resample_period).mean()

dff = dff.reset_index(names="date")

fig = px.line(
    dff,
    x="date",
    y=years,
    hover_data={"date": "|%d. %B, %H:%M"},
#     color_discrete_sequence=px.colors.sequential.Blues,
    color_discrete_sequence=sns.color_palette("mako",len(dff.columns)).as_hex()
)
fig.update_traces(opacity=0.5)

x = list(dff["date"])
y = dff.mean(axis=1).values

y_upper = y + dff.std(axis=1).values
y_lower = y - dff.std(axis=1).values

fig.add_traces(
    [
        go.Scatter(
            x=np.concatenate((x, x[::-1])),  # x, then x reversed
            y=np.concatenate((y_upper, y_lower[::-1])),  # upper, then lower reversed
            fill="toself",
            fillcolor="#A9B7C7",
            line=dict(color="#A9B7C7"),
            hoverinfo="skip",
            showlegend=False,
            name="std",
        ),
        go.Scatter(x=x, y=y, name="Mean", line=dict(color="#3a4454"), mode="lines"),
    ]
)

fig.update_xaxes(
    dtick="M1",
    tickformat="%j",
    ticklabelmode="period"
)

my_template.layout.legend = dict(yanchor=None, xanchor=None,# y = 0.95, x=1.2,
                              bgcolor='rgba(255, 255, 255, 0.8)', bordercolor='black', borderwidth=1,
                                font=dict(size=12, family='Times New Roman'))
fig.update_layout(
    title=f"Area: {area}, with resample period of {resample_period}",
    template=my_template,
    xaxis_title="Day",
    yaxis_title="Wind power output [-]",
)

if resample_period == '1H':
    viz_cols = ["Mean", "std"]
else:
    viz_cols = ["Mean", "std", "2010", "2013"]

fig.update_traces(visible="legendonly", selector=lambda t: not t.name in viz_cols)

fig.update_xaxes(
    dtick="M1",
    tickformat="%b")

fig.show()

In [ ]:
fig.write_image(f"../images/{area}-std-wind-{resample_period}.pdf")

In [ ]:
dff

# Distributions

In [ ]:
from sklearn.neighbors import KernelDensity

N=50
n_scatter_samples=500
bandwidth=0.1
rtol=0.01
kernel="gaussian"
z_max = 2.85

area_a="Sørlige Nordsjø II"
area_b="Nordmela"

area_a="Sørlige Nordsjø II"
area_b="DE West"

x = np.linspace(0.0, 1.0, N)
y = np.linspace(0.0, 1.0, N)
X, Y = np.meshgrid(x, y)
positions = np.vstack([Y.ravel(), X.ravel()])

values = np.vstack([df[area_a].values, df[area_b].values])

# Using scikit learn
kde = KernelDensity(bandwidth=bandwidth, rtol=rtol, kernel=kernel).fit(values.T)
Z = np.reshape(np.exp(kde.score_samples(positions.T)), X.shape)

dff = df.sample(n_scatter_samples)

data = [
    go.Contour(
        z=Z,
        x=x,
        y=y,
        zmax=z_max,
        zauto=False,
        colorscale="Blues",
        # reversescale=True,
        opacity=0.9,
        contours=go.contour.Contours(showlines=False),
        colorbar=dict(lenmode="fraction", len=0.9, y=0.42),
    ),
    go.Scatter(
        x=dff[area_b],
        y=dff[area_a],
        mode="markers",
        marker=dict(line_width=0, opacity=0.3, color="#778DA9", symbol="x"),
    ),
    go.Histogram(
        x=df[area_b].values, name=f"x ", yaxis="y2", histnorm="probability density", marker_color="rgb(220,220,220)"
    ),
    go.Histogram(
        y=df[area_a].values, name=f"y ", xaxis="x2", histnorm="probability density", marker_color="rgb(220,220,220)"
    ),
]

layout = go.Layout(
    # title="",
    # font=go.layout.Font(family="Georgia, serif", color="#635F5D"),
    showlegend=False,
    autosize=False,
    width=650,
    height=650,
    xaxis=dict(domain=[0, 0.85], range=[0, 1], showgrid=False, nticks=7, title=area_b, zeroline=False),
    yaxis=dict(domain=[0, 0.85], range=[0, 1], showgrid=False, nticks=7, title=area_a),
    margin=go.layout.Margin(l=20, r=20, b=20, t=20),
    xaxis2=dict(domain=[0.87, 1], showgrid=False, nticks=7, title="", visible=False),
    yaxis2=dict(domain=[0.87, 1], showgrid=False, nticks=7, title="", visible=False),
    # paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor="rgb(255,255,255)",
)

fig = go.Figure(data=data, layout=layout)

fig.show()

In [ ]:
def latinfy(string):
    return string.replace("æ","ae").replace("ø","oe").replace("å","aa")

In [ ]:
fig.write_image(f"../images/scatter-{latinfy(area_a)}-{latinfy(area_b)}.pdf")

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z, x=x, y=y)])
fig.update_layout(title='', autosize=False,
                  width=1000, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90))